<a href="https://colab.research.google.com/github/Erickrus/llm/blob/main/graphrag_with_ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GraphRAG with Ollama

<img src="https://miro.medium.com/v2/resize:fit:1400/format:webp/0*gY6TXGpdSoUomOTs.png" width=500px />

tutorial: https://www.youtube.com/watch?v=BLyGDTNdad0

url: https://microsoft.github.io/graphrag/

paper: https://arxiv.org/pdf/2404.16130

github: https://github.com/microsoft/graphrag

In [ ]:
!nvidia-smi

Thu Jul 25 11:54:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P8              12W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## Install Ollama

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
#@title select model
import os
MODEL_NAME = "llama3.1:8b" #@param ["wangshenzhi/llama3-8b-chinese-chat-ollama-q8", "phi3", "llama3.1:8b", "llava", "gemma2"]
os.environ["MODEL_NAME"] = MODEL_NAME

In [ ]:
!pip3 install -q ollama
# ollama==0.3.0
!pip3 install -q --upgrade ollamax

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.3 MB/s eta 0:00:00
ERROR: Could not find a version that satisfies the requirement ollamax (from versions: none)
ERROR: No matching distribution found for ollamax


In [ ]:
#@title start ollama as a service
!nohup ollama serve &
#!echo 'hello' |ollama run {MODEL_NAME}

nohup: appending output to 'nohup.out'


In [ ]:
!ps -ef | grep ollama

root        1193       1  6 10:46 ?        00:01:58 ollama serve
root        3565    1193 72 10:54 ?        00:14:29 /tmp/ollama2480822156/runners/cuda_v11/ollama_ll
root        8692     733  0 11:14 ?        00:00:00 /bin/bash -c ps -ef | grep ollama
root        8694    8692  0 11:14 ?        00:00:00 grep ollama


In [ ]:
#@title pull models
#@markdown llama3.1:8b

#@markdown nomic-embed-text:v1.5
!ollama pull llama3.1:8b
#!ollama pull gemma2
!ollama pull nomic-embed-text:v1.5

## Install graphrag

In [ ]:
!pip3 install -U -q graphrag
# graphrag==0.2.0

In [ ]:
#@title prepare input/book.txt
#@markdown The Project Gutenberg eBook of A Christmas Carol, by Charles Dickens

#@markdown trim to 500 lines

import os
from bs4 import BeautifulSoup
from urllib.request import urlopen

os.makedirs("/content/input", exist_ok=True)

url = "https://www.gutenberg.org/files/46/46-h/46-h.htm"
html = urlopen(url)

lines = BeautifulSoup(html, "html.parser").get_text()
with open("/content/input/book.txt", "w") as f:
    i = 0
    for line in lines.split("\n"):
        if line.strip() == "":
            continue
        f.write(line.strip()+"\n")
        i += 1
        if i > 500:
            break

https://mer.vin/2024/07/graphrag-ollama/

In [ ]:
#@title .env
%%writefile .env
GRAPHRAG_API_KEY=ollama

Writing .env


In [ ]:
#@title settings.yaml
%%writefile settings.yaml
encoding_model: cl100k_base
skip_workflows: []
llm:
  api_key: ${GRAPHRAG_API_KEY}
  type: openai_chat # or azure_openai_chat
  model: "llama3.1:8b"
  model_supports_json: true # recommended if this is available for your model.
  # max_tokens: 4000
  # request_timeout: 180.0
  api_base: http://localhost:11434/v1
  # api_version: 2024-02-15-preview
  # organization: <organization_id>
  # deployment_name: <azure_model_deployment_name>
  # tokens_per_minute: 150_000 # set a leaky bucket throttle
  # requests_per_minute: 10_000 # set a leaky bucket throttle
  # max_retries: 10
  # max_retry_wait: 10.0
  # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
  # concurrent_requests: 25 # the number of parallel inflight requests that may be made

parallelization:
  stagger: 0.3
  # num_threads: 50 # the number of threads to use for parallel processing

async_mode: threaded # or asyncio

embeddings:
  ## parallelization: override the global parallelization settings for embeddings
  async_mode: threaded # or asyncio
  llm:
    api_key: ${GRAPHRAG_API_KEY}
    type: openai_embedding # or azure_openai_embedding
    model: nomic-embed-text:v1.5
    api_base: http://localhost:11434/v1
    # api_version: 2024-02-15-preview
    # organization: <organization_id>
    # deployment_name: <azure_model_deployment_name>
    # tokens_per_minute: 150_000 # set a leaky bucket throttle
    # requests_per_minute: 10_000 # set a leaky bucket throttle
    # max_retries: 10
    # max_retry_wait: 10.0
    # sleep_on_rate_limit_recommendation: true # whether to sleep when azure suggests wait-times
    # concurrent_requests: 25 # the number of parallel inflight requests that may be made
    # batch_size: 16 # the number of documents to send in a single request
    # batch_max_tokens: 8191 # the maximum number of tokens to send in a single request
    # target: required # or optional



chunks:
  size: 300
  overlap: 100
  group_by_columns: [id] # by default, we don't allow chunks to cross documents

input:
  type: file # or blob
  file_type: text # or csv
  base_dir: "input"
  file_encoding: utf-8
  file_pattern: ".*\\.txt$"

cache:
  type: file # or blob
  base_dir: "cache"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

storage:
  type: file # or blob
  base_dir: "output/${timestamp}/artifacts"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

reporting:
  type: file # or console, blob
  base_dir: "output/${timestamp}/reports"
  # connection_string: <azure_blob_storage_connection_string>
  # container_name: <azure_blob_storage_container_name>

entity_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/entity_extraction.txt"
  entity_types: [organization,person,geo,event]
  max_gleanings: 0

summarize_descriptions:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/summarize_descriptions.txt"
  max_length: 500

claim_extraction:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  # enabled: true
  prompt: "prompts/claim_extraction.txt"
  description: "Any claims or facts that could be relevant to information discovery."
  max_gleanings: 0

community_reports:
  ## llm: override the global llm settings for this task
  ## parallelization: override the global parallelization settings for this task
  ## async_mode: override the global async_mode settings for this task
  prompt: "prompts/community_report.txt"
  max_length: 2000
  max_input_length: 8000

cluster_graph:
  max_cluster_size: 10

embed_graph:
  enabled: false # if true, will generate node2vec embeddings for nodes
  # num_walks: 10
  # walk_length: 40
  # window_size: 2
  # iterations: 3
  # random_seed: 597832

umap:
  enabled: false # if true, will generate UMAP embeddings for nodes

snapshots:
  graphml: false
  raw_entities: false
  top_level_nodes: false

local_search:
  # text_unit_prop: 0.5
  # community_prop: 0.1
  # conversation_history_max_turns: 5
  # top_k_mapped_entities: 10
  # top_k_relationships: 10
  # max_tokens: 12000

global_search:
  # max_tokens: 12000
  # data_max_tokens: 12000
  # map_max_tokens: 1000
  # reduce_max_tokens: 2000
  # concurrency: 32

## Indexing using GraphRag

In [ ]:
#@title graphrag.index init current directory
#@markdown -m graphrag.index is the python module entrance
!python3 -m graphrag.index --init --root .

2024-07-25 11:21:50.846749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 11:21:50.846799: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 11:21:50.968132: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 11:21:53.926565: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Initializing project at .
⠋ GraphRAG Indexer 

In [ ]:
#@title graphrag.index start index
#@markdown this takes very long time, about 28 min on T4

import datetime
print(datetime.datetime.now())
!python3 -m graphrag.index --root . | tee -a graphrag.log
print(datetime.datetime.now())

2024-07-25 11:39:49.334321
2024-07-25 11:39:54.264163: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 11:39:54.264209: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 11:39:54.265665: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 11:39:55.516338: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
🚀 Reading settings from settings.yaml
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'Dat

2024-07-25 11:22:49.012210

## Run GraphRag query

In [ ]:
#@title run -m graphrag.query with prompt
import datetime
print(datetime.datetime.now())
PROMPT="What are the top themes in this story?" #@param {type: "string"}
!python3 -m graphrag.query --root . --method global "What are the top themes in this story?"
print(datetime.datetime.now())

2024-07-25 11:43:48.539838
2024-07-25 11:43:54.539133: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 11:43:54.539181: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 11:43:54.540545: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 11:43:55.694209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


INFO: Reading settings from settings.yaml
creating llm client with {'api_key': 'REDACTED,len=9', 'type': "openai_chat", 'model': 'llama3.1:8b', 'max_tokens': 4000, 'temperature': 0.0, 'top_p': 1.0, 'n': 1, 'reque

In [ ]:
!tree output/20240725-114005

output/20240725-114005
├── artifacts
│   ├── create_base_documents.parquet
│   ├── create_base_entity_graph.parquet
│   ├── create_base_extracted_entities.parquet
│   ├── create_base_text_units.parquet
│   ├── create_final_communities.parquet
│   ├── create_final_community_reports.parquet
│   ├── create_final_documents.parquet
│   ├── create_final_entities.parquet
│   ├── create_final_nodes.parquet
│   ├── create_final_relationships.parquet
│   ├── create_final_text_units.parquet
│   ├── create_summarized_entities.parquet
│   ├── join_text_units_to_entity_ids.parquet
│   ├── join_text_units_to_relationship_ids.parquet
│   └── stats.json
└── reports
    ├── indexing-engine.log
    └── logs.json

2 directories, 17 files




[default_workflows: WorkflowDefinitions](https://github.com/microsoft/graphrag/blob/61b5eea34783c58074b3c53f1689ad8a5ba6b6ee/graphrag/index/workflows/default_workflows.py#L104)

```python
default_workflows: WorkflowDefinitions = {
    create_base_extracted_entities: build_create_base_extracted_entities_steps,
    create_base_entity_graph: build_create_base_entity_graph_steps,
    create_base_text_units: build_create_base_text_units_steps,
    create_final_text_units: build_create_final_text_units,
    create_final_community_reports: build_create_final_community_reports_steps,
    create_final_nodes: build_create_final_nodes_steps,
    create_final_relationships: build_create_final_relationships_steps,
    create_final_documents: build_create_final_documents_steps,
    create_final_covariates: build_create_final_covariates_steps,
    create_base_documents: build_create_base_documents_steps,
    create_final_entities: build_create_final_entities_steps,
    create_final_communities: build_create_final_communities_steps,
    create_summarized_entities: build_create_summarized_entities_steps,
    join_text_units_to_entity_ids: join_text_units_to_entity_ids_steps,
    join_text_units_to_covariate_ids: join_text_units_to_covariate_ids_steps,
    join_text_units_to_relationship_ids: join_text_units_to_relationship_ids_steps,
}
```



```
source documents
  text extraction
  chunking
text chunks
  domain-tailored summarization
element instances
  domain-tailed summarization
element summaries
  community detection
graph communities
  domain-tailored summarization
community summaries
  query-focused summarization
community answers
  query-focused summarization
global answer
```



In [ ]:
!ls -lrt /content/output/20240725-114005/artifacts

total 860
-rw-r--r-- 1 root root  34853 Jul 25 11:40 create_base_text_units.parquet
-rw-r--r-- 1 root root  14900 Jul 25 11:40 create_base_extracted_entities.parquet
-rw-r--r-- 1 root root  15321 Jul 25 11:40 create_summarized_entities.parquet
-rw-r--r-- 1 root root  21220 Jul 25 11:40 create_base_entity_graph.parquet
-rw-r--r-- 1 root root  27645 Jul 25 11:40 create_final_nodes.parquet
-rw-r--r-- 1 root root 549666 Jul 25 11:40 create_final_entities.parquet
-rw-r--r-- 1 root root   8536 Jul 25 11:40 join_text_units_to_entity_ids.parquet
-rw-r--r-- 1 root root  17108 Jul 25 11:40 create_final_relationships.parquet
-rw-r--r-- 1 root root   7735 Jul 25 11:40 create_final_communities.parquet
-rw-r--r-- 1 root root   4769 Jul 25 11:40 join_text_units_to_relationship_ids.parquet
-rw-r--r-- 1 root root  55819 Jul 25 11:43 create_final_community_reports.parquet
-rw-r--r-- 1 root root  38728 Jul 25 11:43 create_final_text_units.parquet
-rw-r--r-- 1 root root  24417 Jul 25 11:43 create_base_doc

In [ ]:
import pandas as pd
parquet_filename = 'output/20240725-114005/artifacts/create_final_nodes.parquet'
!ls -al {parquet_filename}
pd.read_parquet(parquet_filename, engine='pyarrow')
#df["entity_graph"][0]

-rw-r--r-- 1 root root 27645 Jul 25 11:40 output/20240725-114005/artifacts/create_final_nodes.parquet


,level,title,type,description,source_id,degree,human_readable_id,id,size,graph_embedding,entity_type,community,top_level_node_id,x,y
0,0,"""THE TEAM""",,,73f2b8277b757d6a2b442ee4bd965298,2,0,b45241d70f0e43fca764df95b2b81f77,2,None,None,None,b45241d70f0e43fca764df95b2b81f77,0,0
1,0,"""WASHINGTON""",,,73f2b8277b757d6a2b442ee4bd965298,1,1,4119fd06010c494caa07f439b333f4c5,1,None,None,None,4119fd06010c494caa07f439b333f4c5,0,0
2,0,"""OPERATION: DULCE""",,"""Operation: Dulce is an event in which the tea...","07cdc237557005aa7765bfec48afaf0d,0ebaccb6eaa8c...",1,2,d3835bf3dda84ead99deadbeac5d0d7d,1,None,"""EVENT""",None,d3835bf3dda84ead99deadbeac5d0d7d,0,0
3,0,"""ALEX""","""PERSON""","""Alex is the leader of a team attempting first...",73f2b8277b757d6a2b442ee4bd965298,1,3,077d2820ae1845bcbb1803379a3d1eae,1,None,None,None,077d2820ae1845bcbb1803379a3d1eae,0,0
4,0,"""CONTROL""","""CONCEPT""","""Control refers to the ability to manage or go...",73f2b8277b757d6a2b442ee4bd965298,0,4,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,None,None,None,3671ea0dd4e84c1a9b02c5ab2c8f4bac,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,0,"""DARKNESS""",,,a1adea5ed35f53cc1827586dbd174eb2,1,64,32e6ccab20d94029811127dbbe424c64,1,None,None,0,32e6ccab20d94029811127dbbe424c64,0,0
65,0,"""EBENEZER SCROOGE""","""PERSON""","""Ebenezer Scrooge is a character who locks him...",82568833b02b79dbee4d1881b81954cf,0,65,94a964c6992945ebb3833dfdfdc8d655,0,None,None,None,94a964c6992945ebb3833dfdfdc8d655,0,0
66,0,"""MARLEY'S GHOST""","""PERSON""","""Marley's Ghost is a supernatural entity that ...",a2d9dcfad15df9d83ba21b1ad516c5b9,0,66,1eb829d0ace042089f0746f78729696c,0,None,None,None,1eb829d0ace042089f0746f78729696c,0,0
67,0,"""JACOB MARLEY""","""PERSON""","""Jacob Marley is a person who was Scrooge's bu...",0ebaccb6eaa8ca248bbedfc33942a3a5,1,67,015e7b58d1a14b44beab3bbc9f912c18,1,None,None,0,015e7b58d1a14b44beab3bbc9f912c18,0,0


In [ ]:
!unzip graphrag_data.zip

In [ ]:
!apt-get install openjdk-18-jdk-headless -qq > /dev/null
!java -version
!curl -O https://dist.neo4j.org/neo4j-community-5.21.2-unix.tar.gz
!tar -xf neo4j-community-5.21.2-unix.tar.gz

In [ ]:
!cd neo4j-community-5.21.2 && ./bin/neo4j start
!cd neo4j-community-5.21.2 && ./bin/neo4j status

Directories in use:
home:         /content/neo4j-community-5.21.2
config:       /content/neo4j-community-5.21.2/conf
logs:         /content/neo4j-community-5.21.2/logs
plugins:      /content/neo4j-community-5.21.2/plugins
import:       /content/neo4j-community-5.21.2/import
data:         /content/neo4j-community-5.21.2/data
certificates: /content/neo4j-community-5.21.2/certificates
licenses:     /content/neo4j-community-5.21.2/licenses
run:          /content/neo4j-community-5.21.2/run
Starting Neo4j.
WARNING! You are using an unsupported Java runtime.
* Please use Java(TM) 17 or Java(TM) 21 to run Neo4j.
* Please see https://neo4j.com/docs/ for Neo4j installation instructions.
Started neo4j (pid:2303). It is available at http://localhost:7474
There may be a short delay until the server is ready.
Neo4j is running at pid 2303


In [ ]:
!wget -Ocpolar-stable-linux-amd64.zip https://www.cpolar.com/static/downloads/releases/3.3.18/cpolar-stable-linux-amd64.zip?_gl=1*fmjrcv*_ga*ODE3OTg2MTMwLjE3MjE3ODk5MTY.*_ga_WF16DPKZZ1*MTcyMjA0MjU2My42LjEuMTcyMjA0MjU4My40MC4wLjA.
!unzip cpolar-stable-linux-amd64.zip
!chmod 777 cpolar
!rm -rf cpolar-stable-linux-amd64.zip
!./cpolar authtoken ...

In [ ]:
!nohup ./cpolar http 7474 &

In [ ]:
!ps -ef | grep cpolar

root        2774       1 20 01:43 ?        00:04:36 cpolar: master procSHELL=/bin/bash
root        8254     624  0 02:06 ?        00:00:00 /bin/bash -c ps -ef | grep cpolar
root        8256    8254  0 02:06 ?        00:00:00 grep cpolar


In [ ]:
!kill -9 2774

In [ ]:
#@title auto_domain.py
import IPython

import json
import requests

class Cpolar:
    def get_url(self):
        publicUrl, localAddr = "", ""
        try:
            resp = requests.get("http://127.0.0.1:4040/http/in")
            lines = resp.text.split("\n")
            data = {}
            for line in lines:
                if line.find("window.data = JSON.parse")>0:
                    data = json.loads(line[line.find("window.data = JSON.parse")+25:-2])
                    data = json.loads(data)
                    break
            if len(data.keys()) > 0:
                publicUrl = data["UiState"]["Tunnels"][0]["PublicUrl"]
                localAddr = data["UiState"]["Tunnels"][0]["LocalAddr"]

                publicUrl = publicUrl[publicUrl.rfind('/')+1:]
                localAddr = localAddr[localAddr.rfind('/')+1:]
        except:
            import traceback
            traceback.print_exc()
            pass
        return publicUrl, localAddr
print(Cpolar().get_url())

('3f30525f.r19.vip.cpolar.cn', 'localhost:7474')


In [ ]:
import os
import pandas as pd
import csv

def clean_quotes(value):
    if isinstance(value, str):
        # Remove extra quotes and strip leading/trailing spaces
        value = value.strip().replace('""', '"').replace('"', '')
        # Ensure proper quoting for fields with commas or quotes
        if ',' in value or '"' in value:
            value = f'"{value}"'
    return value

def convert(parquet_dir, csv_dir):
    # Convert all Parquet files to CSV
    for file_name in os.listdir(parquet_dir):
        if file_name.endswith('.parquet'):
            parquet_file = os.path.join(parquet_dir, file_name)
            csv_file = os.path.join(csv_dir, file_name.replace('.parquet', '.csv'))

            # Load the Parquet file
            df = pd.read_parquet(parquet_file)

            # Clean quotes in string fields
            for column in df.select_dtypes(include=['object']).columns:
                df[column] = df[column].apply(clean_quotes)

            # Save to CSV
            df.to_csv(csv_file, index=False, quoting=csv.QUOTE_NONNUMERIC)
            print(f"Converted {parquet_file} to {csv_file} successfully.")

    print("All Parquet files have been converted to CSV.")

parquet_dir = '/content/output/20240725-114005/artifacts'
csv_dir = 'neo4j-community-5.21.2/import'
convert(parquet_dir, csv_dir)

Converted /content/output/20240725-114005/artifacts/create_summarized_entities.parquet to neo4j-community-5.21.2/import/create_summarized_entities.csv successfully.
Converted /content/output/20240725-114005/artifacts/create_base_extracted_entities.parquet to neo4j-community-5.21.2/import/create_base_extracted_entities.csv successfully.
Converted /content/output/20240725-114005/artifacts/create_final_nodes.parquet to neo4j-community-5.21.2/import/create_final_nodes.csv successfully.
Converted /content/output/20240725-114005/artifacts/create_final_communities.parquet to neo4j-community-5.21.2/import/create_final_communities.csv successfully.
Converted /content/output/20240725-114005/artifacts/join_text_units_to_entity_ids.parquet to neo4j-community-5.21.2/import/join_text_units_to_entity_ids.csv successfully.
Converted /content/output/20240725-114005/artifacts/create_final_community_reports.parquet to neo4j-community-5.21.2/import/create_final_community_reports.csv successfully.
Converte

In [ ]:
!curl localhost:7474

{"bolt_routing":"neo4j://localhost:7687","transaction":"http://localhost:7474/db/{databaseName}/tx","bolt_direct":"bolt://localhost:7687","neo4j_version":"5.21.2","neo4j_edition":"community"}